<a href="https://colab.research.google.com/github/sbjglitch/Sanskrit2mal/blob/main/sanskrit_to_malayalam_NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

#%pip install inltk
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [27]:
%cd /Users/Flora Johnson/Desktop/Sanskrit2mal/parallel-corpus

C:\Users\Flora Johnson\Desktop\Sanskrit2mal\parallel-corpus


In [28]:
%cd /Users/Flora Johnson/Desktop/Sanskrit2mal/parallel-corpus
mal_lines = []
sanskrit_lines = []
with open ('bhagavadgita_malayalam.txt','r', encoding="utf8") as f:
  mal_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open ('bhagvadgita_sanskrit.txt','r',encoding="utf8") as f:
  sanskrit_lines.extend([x.replace('\n', '') for x in f.readlines()])

  print(len(mal_lines))
  print(len(sanskrit_lines))

C:\Users\Flora Johnson\Desktop\Sanskrit2mal\parallel-corpus
646
701


In [29]:

c = list(zip(sanskrit_lines, mal_lines))
random.shuffle(c)

sanskrit_lines, mal_lines = zip(*c)

train_text_sa = sanskrit_lines[:-323]
train_text_mal = mal_lines[:-323]

dev_text_sa = sanskrit_lines[-323:]
dev_text_mal = mal_lines[-323:]

In [30]:
print(len(train_text_mal))
print(len(dev_text_mal))

print(len(train_text_sa))
print(len(dev_text_sa))

323
323
323
323


In [31]:
# Start and End tokens
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [24]:
# Reading the data and forming pairs
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Forming pairs of sentences
    pairs = [[train_text_sa[i], train_text_mal[i]] for i in range(len(train_text_sa))]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [8]:
MAX_LENGTH = 200

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [25]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('sanskrit', 'mal', False)
print(random.choice(pairs)) # a random pair of parallel sentences

Reading lines...
Read 272 sentence pairs
Trimmed to 272 sentence pairs
Counting words...
Counted words:
sanskrit 1906
mal 2813
['श्री भगवानुवाच भूय एव महाबाहो श्रृणु मे परमं वचः। यत्तेऽहं प्रीयमाणाय वक्ष्यामि हितकाम्यया।।', 'ദൈത്യന്മാരി\u200d\u200dല്\u200d പ്രഹ്ലാദനും, കണക്കെടുക്കന്നവരില്\u200d കാലനും ഞാനാണ്. മൃഗങ്ങളില്\u200d സിംഹവും പക്ഷികളി\u200d\u200dല്\u200d ഗരുഡനും ഞാ\u200dന്\u200d തന്നെ.']


# **Seq2Seq Model**
---
A seq2seq network is a model consisting of two RNNs called the encoder and decoder. The encoder reads an input sequence and outputs a single vector, and the decoder reads that vector to produce an output sequence.

# **Encoder**

In [11]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# **Decoder**

In [12]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# **Training**


In [13]:
import random
def indexesFromSentence(lang, sentence):
    res = []
    for word in sentence.split(' '):
      if word not in lang.word2index.keys():
        res.append(random.choice(list(lang.word2index.values())))    
      else:
        res.append(lang.word2index[word])
    return res

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

# **Training the model**

In [14]:
teacher_forcing_ratio = 0.5
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]

    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [15]:
# Print time elapsed and estimated time remaining given the current time and progress %.
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [16]:
# Each iteration of training
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

# **Evaluation**

In [17]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [18]:
def evaluate_test(encoder, decoder):
  test = []
  %cd /Users/Flora Johnson/Desktop/Sanskrit2mal/parallel-corpus/sanskrit-mal
  with open('test_sanskrit.sa', 'r', encoding="utf8") as f:
    test_text = [x.replace('\n', '') for x in f.readlines()]
  for sa_text in test_text:
    output_words, attentions = evaluate(encoder, decoder, sa_text)
    output_sentence = ' '.join(output_words)
    test.append(output_sentence)
  with open('translated_malayalam.ma', 'w', encoding="utf8") as f:
    for line in test:
        f.write(line + '\n')

In [19]:
def translator(encoder, decoder):
    test_text = 'संस्कृति'

    output_words, attentions = evaluate(encoder, decoder, test_text)
    output_sentence = ' '.join(output_words)
    print(output_sentence)

In [20]:
%cd /Users/Flora Johnson/Desktop/Sanskrit2mal/parallel-corpus/sanskrit-mal

C:\Users\Flora Johnson\Desktop\Sanskrit2mal\parallel-corpus\sanskrit-mal


In [21]:
# defining hidden layer size and the encoder and decoder networks
hidden_size = 1024

encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

# train
trainIters(encoder1, attn_decoder1, 75000, print_every=500)

1m 19s (- 196m 58s) (500 0%) 5.1425
2m 57s (- 219m 15s) (1000 1%) 5.7834
4m 34s (- 224m 22s) (1500 2%) 5.8752
6m 5s (- 222m 16s) (2000 2%) 5.5465
7m 44s (- 224m 23s) (2500 3%) 5.1681
9m 53s (- 237m 17s) (3000 4%) 4.9125
11m 44s (- 239m 58s) (3500 4%) 4.4321
13m 31s (- 240m 3s) (4000 5%) 3.6756
15m 12s (- 238m 10s) (4500 6%) 2.9701
16m 53s (- 236m 27s) (5000 6%) 2.1612
18m 38s (- 235m 28s) (5500 7%) 1.5937
20m 22s (- 234m 16s) (6000 8%) 1.0889
22m 6s (- 232m 55s) (6500 8%) 0.6179
28m 52s (- 280m 31s) (7000 9%) 0.3644
30m 40s (- 276m 2s) (7500 10%) 0.2821
32m 38s (- 273m 18s) (8000 10%) 0.2001
34m 50s (- 272m 34s) (8500 11%) 0.1329
36m 44s (- 269m 23s) (9000 12%) 0.0916
38m 39s (- 266m 33s) (9500 12%) 0.0686
40m 33s (- 263m 39s) (10000 13%) 0.0640
42m 26s (- 260m 43s) (10500 14%) 0.0635
44m 13s (- 257m 20s) (11000 14%) 0.0547
46m 1s (- 254m 10s) (11500 15%) 0.0457
47m 54s (- 251m 33s) (12000 16%) 0.0500
49m 47s (- 248m 59s) (12500 16%) 0.0393
51m 51s (- 247m 20s) (13000 17%) 0.0506
53m 4

KeyboardInterrupt: 

In [ ]:
# saving the trained models
%cd /Users/Flora Johnson/Desktop/Sanskrit2mal
torch.save(encoder1.state_dict(), 'trained_encoder.pth')
torch.save(attn_decoder1.state_dict(), 'trained_decoder.pth')

In [ ]:
evaluate_test(encoder1, attn_decoder1)

In [ ]:
%cd /Users/Flora Johnson/Desktop/Sanskrit2mal/parallel-corpus/sanskrit-mal
output_text=[]
with open ('translated_malayalam.ma','r',encoding="utf8") as f:
  output_text.extend([f.readlines()])
print(output_text)

/home/glitch/project/Sanskrit2mal/parallel-corpus/sanskrit-mal
[['കൃഷ്ണാ! എനിക്ക് അങ്ങയുടെ യോഗത്തേയും ശരിയായി അറിയുന്നവ\u200dന്\u200d നിശ്ചലമായ ഏകാഗ്രനിഷ്ഠയോടുകൂടിയവനാണ്; അതി\u200cല്\u200d\u200d സംശയമില്ല. <EOS>\n', 'ഹേ കൃഷ്ണാ, മനസ്സ് ചഞ്ചലവും ക്ഷുബ്ധവും നിയന്ത്രണ ത്തിനു വഴങ്ങാത്തതും അയവില്ലാത്തതുമാണ്. അതിന്റെ നിയന്ത്രണം വായുവിന്റെതെന്നപോലെ ദുഷ്ക്കരമായി ഞാന്\u200d കരുതുന്നു. <EOS>\n', 'ഹേ അ\u200cര്\u200dജ്ജുനാ! ഈ രണ്ടു മാര്\u200dഗ്ഗങ്ങളെ യഥാ\u200d\u200dര്\u200dത്ഥമായറിയുന്ന യാതൊരു ക\u200dര്\u200dമ്മയോഗിയും മോഹത്തെ പ്രാപിക്കുന്നില്ല.. അതുകൊണ്ടു ഹേ അ\u200cര്\u200dജ്ജുനാ, സമല കാലങ്ങളിലും ക\u200dര്\u200dമ്മയോഗയുക്തനയി ഭവിക്കുക. <EOS>\n', 'എന്റെ അവ്യയവും അനുത്തമവും തമോഗുണസ്പര്\u200dശമില്ലാത്തതുമായ സര്\u200dവാതീതഭാവത്തെ അറിയാതെ അവ്യക്തനായ എന്നെ വ്യക്തിത്വം പ്രാപിച്ചവനെന്നു ബുദ്ധിഹീനര്\u200d വിചാരിക്കുന്നു. <EOS>\n', '[] <EOS>\n', 'ഞാന്\u200d അമൃതവും (അനശ്വരവും) അവ്യയവുമായ (മാറ്റമില്ലാത്തതുമായ) ബ്രഹ്മത്തിന്റെയും, ശാശ്വതമായ ധര്\u200dമ്മ ത്തിന്റെയും, പരമമായ സുഖത്തിന്റെയും പ്രതിഷ്ഠയുമാകുന്നു (ആശ

In [ ]:
translator(encoder1, attn_decoder1)

ഹേ സ‍ര്‍വാത്മ‍ന്‍, അങ്ങനെ മുന്നിലും പിന്നിലും നിന്ന് ഞാ‌ന്‍ നമസ്‍കരിക്കുന്നു. എല്ലാഭാഗത്തും അങ്ങേയ്ക്കു നമസ്‍കാരം. അളവറ്റ വീരവും അതിരറ്റ പരാക്രമവും ഉള്ളവനായ അങ്ങ് ലോകം മുഴുവ‍ന്‍ വ്യാപിച്ചവനായിരിക്കുന്നു. അതിനാല്‍ എല്ല‍ാം അങ്ങ് തന്നെയാകുന്നു. <EOS>


In [ ]:
%cd /Users/Flora Johnson/Desktop/Sanskrit2mal/parallel-corpus/sanskrit-mal

from nltk.translate.bleu_score import sentence_bleu
reference = []
candidate = []

with open ('test_malayalam.ma','r', encoding="utf8") as f:
  reference.extend([x.replace('\n', '') for x in f.readlines()])

with open ('translated_malayalam.ma','r',encoding="utf8") as f:
  candidate.extend([x.replace('\n', '') for x in f.readlines()])


score = sentence_bleu(reference, candidate)
print('bleu score',score)

/home/glitch/project/Sanskrit2mal/parallel-corpus/sanskrit-mal
bleu score 0
